In [62]:
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
np.random.seed(0)
import re

In [4]:
df= pd.read_csv('all_recipe_data.csv')
df.head()

,Recipe_Name,Recipe_Ingredients,Cuisine
0,Bunny chow,2 tbsp vegetable oil ½ tsp cumin seeds ½ tsp f...,african
1,Jollof rice with fried plantains,"1 tbsp olive or vegetable oil 2 large onions, ...",african
2,Jollof rice,400ml/14fl oz passata 3 tbsp tomato purée 2 fr...,african
3,Suya fillet burger with sweet potato cubes and...,4g smoked paprika 2g cayenne pepper 6g ginger...,african
4,Jollof rice with chicken,300g/10½oz basmati rice 1 tbsp vegetable oil 8...,african


In [5]:
df= df.drop_duplicates(subset='Recipe_Name')

In [6]:
df.describe()

,Recipe_Name,Recipe_Ingredients,Cuisine
count,475,475,475
unique,475,475,21
top,Pâte sucrée (sweet shortcrust pastry),450g/1lb lean beef mince 1 tsp crushed garlic...,caribbean
freq,1,1,24


In [12]:
msk = np.random.rand(len(df)) < 0.8

In [23]:
train = df[msk]
len(train)
# train

382

In [24]:
test = df[~msk]
len(test)
# test

93

In [25]:
train.shape

(382, 3)

In [26]:
test.shape

(93, 3)

In [34]:
data_train= train['Recipe_Ingredients']
target_train= train['Cuisine']
data_test= test['Recipe_Ingredients']
target_test= test['Cuisine']
print(data_train.shape)
print(target_train.shape)
print(data_test.shape)
print(target_test.shape)

(382,)
(382,)
(93,)
(93,)


In [80]:
# data_train=re.sub("[^'?a-z\,?]", ' ', train['Recipe_Ingredients'][i])

In [73]:
nltk.download('stopwords')

stopwords_list = stopwords.words('english')+ list(string.punctuation)
stopwords_list+= ["''", '""', '...', '``']
# regex = r"[^'?a-z\,?]"
# stopwords_list

[nltk_data] Downloading package stopwords to /Users/Iffy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
def process_article(article):
    tokens = nltk.word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token not in stopwords_list]
    return stopwords_removed 

In [75]:
# nltk.download('punkt')
processed_train =  list(map(process_article, data_train))
processed_test= list(map(process_article, data_test))

In [76]:
print('cleaned train: ', processed_train[0])
print('----'*28)
print('cleaned test: ', processed_test[0])

cleaned train:  ['2', 'tbsp', 'vegetable', 'oil', '½', 'tsp', 'cumin', 'seeds', '½', 'tsp', 'fennel', 'seeds', '2.5cm/1in', 'piece', 'cinnamon', 'stick', '2', 'green', 'cardamom', 'pods', '1', 'star', 'anise', '1', 'bay', 'leaf', '1', 'onion', 'finely', 'chopped', '2', 'tbsp', 'south', 'african', 'curry', 'powder', '2', 'tomatoes', 'chopped', '1kg/2lb', '2oz', 'boneless', 'leg', 'lamb', 'cut', '1.5cm/½in', 'dices', '1', 'tbsp', 'finely', 'chopped', 'fresh', 'ginger', '1', 'tbsp', 'finely', 'chopped', 'garlic', '10-12', 'curry', 'leaves', '2', 'large', 'potatoes', 'cut', 'cubes', 'size', 'meat', 'salt', '2', 'tbsp', 'finely', 'chopped', 'coriander', 'leaves', '2', 'tbsp', 'lime', 'juice', '2', 'loaves', 'crusty', 'white', 'bread', 'unsliced', 'cut', 'across', 'half', 'middle', 'crumbs', 'removed', 'coriander', 'cress', 'sprigs', 'garnish']
----------------------------------------------------------------------------------------------------------------
cleaned test:  ['butter', 'greasing'

In [109]:
pattern= r"[a-z]+"
p= re.compile(pattern)
ptrain= p.findall(str(processed_train))
len(ptrain)
# print(ptrain)

22501

In [110]:
ptest= p.findall(str(processed_test))
len(ptest)

5376

In [50]:
articles_concat_train = []
for pd in processed_train:
    articles_concat_train += pd
articles_concat_test = []
for pd in processed_test:
    articles_concat_test += pd

In [53]:
articles_freqdist_train = FreqDist(articles_concat_train)
# articles_freqdist_train.most_common(200)
articles_freqdist_test = FreqDist(articles_concat_test)
# articles_freqdist_test.most_common(200)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
vectorizer = TfidfVectorizer()

In [100]:
tf_idf_ptrain = vectorizer.fit_transform(ptrain)
tf_idf_ptest = vectorizer.transform(ptest)
print(tf_idf_ptrain.shape)
print(tf_idf_ptest.shape)

(22501, 1438)
(5376, 1438)


In [101]:
non_zero_cols = tf_idf_ptrain.nnz / float(tf_idf_ptrain.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tf_idf_ptrain.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 0.9434691791475934
Percentage of columns containing 0: 0.9993439018225677


In [102]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier(n_estimators=100)

In [105]:
nb_classifier.fit(tf_idf_ptrain, ptrain)
nb_train_preds = nb_classifier.predict(tf_idf_ptrain)
nb_test_preds = nb_classifier.predict(tf_idf_ptest)

In [106]:
rf_classifier.fit(tf_idf_ptrain, ptrain)
rf_train_preds = rf_classifier.predict(tf_idf_ptrain)
rf_test_preds = rf_classifier.predict(tf_idf_ptest)

In [117]:
nb_train_score = accuracy_score(ptrain, nb_train_preds)
nb_test_score = accuracy_score(ptest, nb_test_preds)
rf_train_score = accuracy_score(ptrain, rf_train_preds)
rf_test_score = accuracy_score(ptest, rf_test_preds)

print("Multinomial Naive Bayes")
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(nb_train_score, nb_test_score))
print("")
print('-'*70)
print("")
print('Random Forest')
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(rf_train_score, rf_test_score))

Multinomial Naive Bayes
Training Accuracy: 0.663 		 Testing Accuracy: 0.6704

----------------------------------------------------------------------

Random Forest
Training Accuracy: 0.9956 		 Testing Accuracy: 0.9706
